## Создание продвинутых ассистентов

В этом ноутбуке мы попробуем создать и протестировать чат-ассистента на основе Yandex Assistant API, RAG и Function Calling.

Для начала, установим Yandex Cloud ML SDK. В идеальном мире, вы сделаете вот так:

In [9]:
%pip install --upgrade --quiet yandex-cloud-ml-sdk

I0000 00:00:1742470000.768423    3331 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: yandex-cloud-ml-sdk
    Found existing installation: yandex-cloud-ml-sdk 0.4.1
    Uninstalling yandex-cloud-ml-sdk-0.4.1:
      Successfully uninstalled yandex-cloud-ml-sdk-0.4.1

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


В нашем случае мы установим самую последнюю версию SDK:

In [1]:
%pip install --quiet -I git+https://github.com/yandex-cloud/yandex-cloud-ml-sdk.git@assistants_fc#egg=yandex-cloud-ml-sdk

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/yandex-cloud/yandex-cloud-ml-sdk.git (to revision assistants_fc) to /tmp/pip-install-n8iaylbn/yandex-cloud-ml-sdk_4769386f056c434493486e0f75e24b69
  Running command git clone --filter=blob:none --quiet https://github.com/yandex-cloud/yandex-cloud-ml-sdk.git /tmp/pip-install-n8iaylbn/yandex-cloud-ml-sdk_4769386f056c434493486e0f75e24b69
  Running command git checkout -b assistants_fc --track origin/assistants_fc
  Switched to a new branch 'assistants_fc'
  Branch 'assistants_fc' set up to track remote branch 'assistants_fc' from 'origin'.
  Resolved https://github.com/yandex-cloud/yandex-cloud-ml-sdk.git to commit 4603be4548f315f3111fe445de91718256a23dbc
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 19.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

Также необходимо обновить некоторые библиотеки:

In [53]:
%pip install --upgrade pydantic

I0000 00:00:1743425416.188163    3524 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
confection 0.1.0 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.11.1 which is incompatible.
inflect 6.0.5 requires pydantic<2,>=1.9.1, but you have pydantic 2.11.1 which is incompatible.
spacy 3.5.4 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.11.1 which is incompatible.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.3 which is incompatible.
thinc 8.1.10 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.11.1 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] T

И ещё пара полезных функций на будущее:

In [71]:
from IPython.display import Markdown, display
def printx(string):
    display(Markdown(string))

Для работы с языковыми моделями нам понадобится ключ `api_key` для сервисного аккаунта, имеющего права на доступ к модели, и `folder_id`. Мы предполагаем, что соответствующие значения хранятся в секретах Datasphere.

Создадим модель последней версии YandexGPT 5 и убедимся, что она кое-что знает про вина:

In [15]:
import os
from yandex_cloud_ml_sdk import YCloudML

folder_id = os.environ["folder_id"]
api_key = os.environ["api_key"]

sdk = YCloudML(folder_id=folder_id, auth=api_key)

# Раскомментируйте, если хотите подробнее смотреть, что делает SDK
#sdk.setup_default_logging(log_level='DEBUG')

model = sdk.models.completions("yandexgpt", model_version="rc")

In [72]:
printx(model.run("Какое вино можно пить с устрицами?").text)

Выбор вина к устрицам зависит от ваших личных предпочтений и вида устриц. Обычно с устрицами подают белое вино, так как оно хорошо сочетается с их солоноватым вкусом и текстурой.

Вот несколько примеров вин, которые часто рекомендуют к устрицам:

1. Шардоне (Chardonnay) — особенно из прохладных климатических зон, таких как Шабли или Калифорния, может быть отличным выбором. Оно обладает свежим вкусом с нотами цитрусовых и зелёного яблока.
2. Совиньон Блан (Sauvignon Blanc) — особенно из Новой Зеландии, часто имеет яркий и свежий вкус с нотами грейпфрута и зелёного перца.
3. Альбариньо (Albarino) — белое вино из региона Риас Байшас в Испании, известное своей свежестью и фруктовостью.
4. Вердело (Verdelho) — португальское белое вино с освежающим вкусом и нотами цитрусовых.
5. Мюскаде (Muscadet) — белое вино из региона Луары во Франции, которое часто рекомендуют к морепродуктам, включая устрицы.

Помните, что выбор вина — это дело вкуса, и вы можете экспериментировать с разными сортами, чтобы найти то, что вам больше всего нравится.

## Assistant API

Для ведения беседы с моделью с сохранением контекста диалога используем Assistants API. Объект `thread` будет отвечать за сохранение контекста, а `assistant` - за все основные установки, связанные с работой ассистента.

In [17]:
def create_thread():
    return sdk.threads.create(ttl_days=1, expiration_policy="static")

def create_assistant(model, tools=None):
    kwargs = {}
    if tools and len(tools) > 0:
        kwargs = {"tools": tools}
    return sdk.assistants.create(
        model, ttl_days=1, expiration_policy="since_last_active", **kwargs
    )

In [4]:
thread = create_thread()
assistant = create_assistant(model)

assistant.update(
    instruction="""Ты - опытный сомелье, задача которого - консультировать пользователя в
    вопросах выбора вина."""
)

thread.write("Привет! Какое вино посоветуете?")

run = assistant.run(thread)
result = run.wait()

print(result.text)

Здравствуйте! Чтобы подобрать наиболее подходящее вино, мне нужно знать несколько деталей:

1. Какой тип вина вы предпочитаете: красное, белое, розовое, игристое или десертное?
2. Какой стиль вина вам нравится: лёгкое и свежее или более насыщенное и крепкое?
3. Есть ли у вас предпочтения по вкусам и ароматам? Например, фруктовые, ягодные, цветочные, минеральные и т. д.
4. В каком ценовом диапазоне вы ищете вино?
5. С какими блюдами вы планируете сочетать вино?

Чем больше информации вы предоставите, тем точнее я смогу подобрать рекомендации.


In [5]:
thread.write("Я буду есть стейк!")

run = assistant.run(thread)
result = run.wait()

print(result.text)

Отлично! Для стейка я могу порекомендовать несколько вариантов красного вина:

1. Каберне Совиньон — это насыщенное вино с яркими фруктовыми нотами и умеренной кислотностью, которое хорошо сочетается с мясом.
2. Мерло — более мягкое и фруктовое вино, которое также хорошо подойдёт к стейку.
3. Шираз (Сира) — это насыщенное вино с нотами тёмного шоколада и специй, которое подчеркнёт вкус мяса.
4. Мальбек — вино с богатым вкусом и ароматом, которое отлично дополнит стейк.
5. Темпранильо — испанское вино с нотами красных фруктов и ванили, которое также может быть хорошим выбором.

Если у вас есть предпочтения по стилю вина или ценовому диапазону, пожалуйста, сообщите мне, и я смогу дать более конкретные рекомендации.


In [6]:
thread.delete()
assistant.delete()

## Добавляем RAG

Для RAG будем использовать текстовую базу знаний по винам и винным регионам, которая хранится в виде множества файлов в директориях `source/wines` и `source/regions`. Пройдёмся по этим файлам и посмотрим на их длину в токенах.

In [7]:
from glob import glob
from tqdm.auto import tqdm
import pandas as pd

def get_token_count(filename):
    with open(filename, "r", encoding="utf8") as f:
        return len(model.tokenize(f.read()))

def get_file_len(filename):
    with open(filename, encoding="utf-8") as f:
        l = len(f.read())
    return l

d = [
    {
        "File": fn,
        "Tokens": get_token_count(fn),
        "Chars": get_file_len(fn),
        "Category": fn.split("/")[1],
    }
    for fn in glob("source/*/*.md")
    if fn.count("/") == 2
]

df = pd.DataFrame(d)
df

,File,Tokens,Chars,Category
0,source/regions/Абруццо.md,499,2022,regions
1,source/regions/Азорские острова.md,409,1809,regions
2,source/regions/Аконкагуа.md,278,1182,regions
3,source/regions/Алентежу.md,320,1216,regions
4,source/regions/Апулия.md,489,1918,regions
...,...,...,...,...
125,source/wines/Совиньон блан.md,634,2578,wines
126,source/wines/Темпранильо.md,637,2323,wines
127,source/wines/Цвайгельт.md,663,2620,wines
128,source/wines/Шардоне.md,567,2407,wines


Посмотрим на среднюю, мин и макс длину фрагментов:

In [8]:
df.groupby("Category").agg({"Tokens": ("min", "mean", "max")})

Tokens                 
            min        mean  max
Category                        
regions     153  423.340000  683
wines       264  551.366667  664

Мы видим, что фрагменты не превышают 700 токенов, и это значит, что нам не придётся прибегать к какой-либо стратегии чанкования. Это идеальная ситуация, когда текстовая база знаний разбита вручную на небольшие фрагменты текста.

## Загружаем файлы в облако

Чтобы RAG мог осущетвлять поиск по фрагментам файлов, нам необходимо построить индекс, а перед этим - загрузить все файлы в облако.

In [9]:
def upload_file(filename):
    return sdk.files.upload(filename, ttl_days=1, expiration_policy="static")

df["Uploaded"] = df["File"].apply(upload_file)

## Строим индекс

Для индексации файлов можно применять следующие стратегии:
* Поиск по эмбеддингам (векторный поиск) - по всем фрагментам текста вычисляются эмбеддинги, и в процессе поиска осуществляется векторный поиск между эмбеддингом запроса и эмбеддингом фрагмента. Это позволяет осуществлять семантический поиск
* Поиск по ключевым словам
* Гибридный поиск, в котором различными способами объединяются результаты поиска по ключевым словам и вектороного поиска.

Также в процессе создания индекса мы указываем **стратегию чанкования** текстов, но в нашем случае она не будет применяться.

> Поскольку есть ограничение на 100 добавляемых в индекс файлов, то будем добавлять фрагменты по винам и по регионам по-очереди.

In [12]:
from yandex_cloud_ml_sdk.search_indexes import (
    StaticIndexChunkingStrategy,
    HybridSearchIndexType,
    ReciprocalRankFusionIndexCombinationStrategy,
)

op = sdk.search_indexes.create_deferred(
    df[df["Category"] == "wines"]["Uploaded"],
    index_type=HybridSearchIndexType(
        chunking_strategy=StaticIndexChunkingStrategy(
            max_chunk_size_tokens=1000, chunk_overlap_tokens=100
        ),
        combination_strategy=ReciprocalRankFusionIndexCombinationStrategy(),
    ),
)
index = op.wait()

[I 2025-04-02 04:16:17 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvtn4fhuhrmhq4vrbgd2 polling with a poll interval 10.000000s and poll timeout 3600.000000s
[D 2025-04-02 04:16:17 yandex_cloud_ml_sdk._types.operation:211] Fetching operation fvtn4fhuhrmhq4vrbgd2 status
[D 2025-04-02 04:16:17 yandex_cloud_ml_sdk._types.operation:225] Operation fvtn4fhuhrmhq4vrbgd2 have status OperationStatus<runnning>
[D 2025-04-02 04:16:17 yandex_cloud_ml_sdk._types.operation:110] Operation fvtn4fhuhrmhq4vrbgd2 have non-terminal status runnning, sleep for 10.000000s
[D 2025-04-02 04:16:27 yandex_cloud_ml_sdk._types.operation:211] Fetching operation fvtn4fhuhrmhq4vrbgd2 status
[D 2025-04-02 04:16:27 yandex_cloud_ml_sdk._types.operation:225] Operation fvtn4fhuhrmhq4vrbgd2 have status OperationStatus<success>
[I 2025-04-02 04:16:27 yandex_cloud_ml_sdk._types.operation:118] Operation fvtn4fhuhrmhq4vrbgd2 successfully finished
[I 2025-04-02 04:16:27 yandex_cloud_ml_sdk._types.operation:142]

In [14]:
op = index.add_files_deferred(df[df["Category"]=="regions"]["Uploaded"])
xfiles = op.wait()

[I 2025-04-02 04:16:53 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvttj82q8egus4akhe0j polling with a poll interval 10.000000s and poll timeout 3600.000000s
[D 2025-04-02 04:16:53 yandex_cloud_ml_sdk._types.operation:211] Fetching operation fvttj82q8egus4akhe0j status
[D 2025-04-02 04:16:53 yandex_cloud_ml_sdk._types.operation:225] Operation fvttj82q8egus4akhe0j have status OperationStatus<runnning>
[D 2025-04-02 04:16:53 yandex_cloud_ml_sdk._types.operation:110] Operation fvttj82q8egus4akhe0j have non-terminal status runnning, sleep for 10.000000s
[D 2025-04-02 04:17:03 yandex_cloud_ml_sdk._types.operation:211] Fetching operation fvttj82q8egus4akhe0j status
[D 2025-04-02 04:17:03 yandex_cloud_ml_sdk._types.operation:225] Operation fvttj82q8egus4akhe0j have status OperationStatus<success>
[I 2025-04-02 04:17:03 yandex_cloud_ml_sdk._types.operation:118] Operation fvttj82q8egus4akhe0j successfully finished
[I 2025-04-02 04:17:03 yandex_cloud_ml_sdk._types.operation:142]

## Собираем RAG-ассистента

Теперь собираем собственно ассистента, который будет использовать RAG. Для этого определяем **инструмент** (tool) для поиска в нашем индексе, и указываем его при создании ассистента. Также важно задать хорошую инструкцию для ассистента (системный промпт): 

In [18]:
search_tool = sdk.tools.search_index(index)

assistant = create_assistant(model, tools=[search_tool])
thread = create_thread()

instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде. Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций. Если что-то непонятно, то лучше уточни информацию
у пользователя.
"""

_ = assistant.update(instruction=instruction)

In [19]:
thread.write("Какое вино подходит к стейку?")
run = assistant.run(thread)

result = run.wait()
print(result.text)

[I 2025-04-02 04:19:15 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvt7l8ep6qb52rnrqmgc polling with a poll interval 0.500000s and poll timeout 300.000000s
[D 2025-04-02 04:19:15 yandex_cloud_ml_sdk._types.operation:110] Operation fvt7l8ep6qb52rnrqmgc have non-terminal status PENDING, sleep for 0.500000s
[D 2025-04-02 04:19:16 yandex_cloud_ml_sdk._types.operation:110] Operation fvt7l8ep6qb52rnrqmgc have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 04:19:17 yandex_cloud_ml_sdk._types.operation:110] Operation fvt7l8ep6qb52rnrqmgc have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 04:19:17 yandex_cloud_ml_sdk._types.operation:110] Operation fvt7l8ep6qb52rnrqmgc have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 04:19:18 yandex_cloud_ml_sdk._types.operation:110] Operation fvt7l8ep6qb52rnrqmgc have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 04:19:18 yandex_cloud_ml_sdk._types.operation:1

К стейку могут подойти следующие вина:

1. **Пинотаж из Франшхука (ЮАР)** — универсальное вино, которое хорошо сочетается с различными блюдами, включая перчёные стейки из говядины и баранины.
2. **Черасуоло ди Виттория (Сицилия, Италия)** — гармонирует с тушёной говядиной и мясом на вертеле.
3. **Красное сухое вино из долины Маллеко (Чили, регион Сур)** — хорошо сочетается со стейком на гриле.
4. **Пино нуар из Орегона (США)** — будет хорошо сочетаться со стейком из лосося и красным мясом, например, говядиной по-бургундски.
5. **Сира или шираз** — благодаря сильному характеру и содержанию танинов, особенно хорошо дополняют блюда из красного мяса, например, ростбиф или мясные рулеты.


Посмотрим, из каких источников был получен этот ответ:

In [20]:
def print_citations(result):
    for citation in result.citations:
        for source in citation.sources:
            if source.type != "filechunk":
                continue
            print("------------------------")
            print(source.parts[0])

print_citations(result)

------------------------
## Франшхук 

Франшхук
Франшхук — крупная винодельческая провинция Южной Африки. Территориально относится к региону Стелленбош, расположена в 75 км от Кейптауна — столицы ЮАР. Деревню основали беглые французы в 1688 году. Получив земельные наделы, они разбили виноградники и наладили производство напитков. Сегодня Франшхук называют винной столицей ЮАР. На карте провинции можно насчитать 11 крупных винодельческих хозяйств. Деревня Франшхук известна необычным терруаром: она расположена в продолговатой долине и с трех сторон окружена высокими горами, которые защищают виноградники от ветра, излишней влаги зимой и палящего солнца летом. При этом четвертая сторона долины открыта для ветров Атлантики. Климат здесь умеренный, а температура воздуха ниже, чем в соседних областях. Виноград созревает медленнее, чем на открытой африканской местности, что делает вино более свежим и кислотным. В Франшхуке наиболее распространены известные европейские сорта, завезенные француза

In [21]:
thread.delete()

Мы видим, что ответ получился несколько однобоким, поскольку данные о сочетании вин и еды содержатся в текстовой базе знаний в разрозненном виде.

## Добавляем таблицу соответствий

Поскольку подбор блюда к вину является частой задачей, добавим к нашей базе знаний явную табличку соответствий блюд и вин, которая находится в файле `source/food_wine_table.md` в формате markdown.

In [22]:
with open("source/food_wine_table.md", encoding="utf-8") as f:
    food_wine = f.readlines()
fw = "".join(food_wine)

tokens = len(model.tokenize(fw))
print(f"Токенов: {tokens}, {len(fw)/tokens} chars/token")

Токенов: 12630, 3.3103721298495645 chars/token


Видим, что табличка большая, поэтому её придётся *чанковать*. Но при этом важно чанковать табличку так, чтобы в каждом фрагмента оставался заголовок таблицы, который определяет семантику столбцов.

Отделим заголовок таблицы:

In [23]:
header = food_wine[:2]
header

['Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду\n',
 '--------|--------\n']

Ниже будем чанковать табличку вручную, задав размер чанка в символах для простоты. Мы будем сразу загружать получившиеся фрагменты в облако, минуя диск:

In [24]:
chunk_size = 600 * 3  # approx 600 tokens * 2 char/token

s = header.copy()
uploaded_foodwine = []
for x in food_wine[2:]:
    s.append(x)
    if len("".join(s)) > chunk_size:
        id = sdk.files.upload_bytes(
            "".join(s).encode(), ttl_days=5, expiration_policy="static"
        )
        uploaded_foodwine.append(id)
        s = header.copy()
print(f"Uploaded {len(uploaded_foodwine)} table chunks")

Uploaded 22 table chunks


Теперь добавим эти фрагменты в индекс:

In [25]:
op = index.add_files_deferred(uploaded_foodwine)
xfiles = op.wait()

[I 2025-04-02 04:20:31 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvtk4abrkk7fsc772lq6 polling with a poll interval 10.000000s and poll timeout 3600.000000s
[D 2025-04-02 04:20:31 yandex_cloud_ml_sdk._types.operation:211] Fetching operation fvtk4abrkk7fsc772lq6 status
[D 2025-04-02 04:20:31 yandex_cloud_ml_sdk._types.operation:225] Operation fvtk4abrkk7fsc772lq6 have status OperationStatus<runnning>
[D 2025-04-02 04:20:31 yandex_cloud_ml_sdk._types.operation:110] Operation fvtk4abrkk7fsc772lq6 have non-terminal status runnning, sleep for 10.000000s
[D 2025-04-02 04:20:41 yandex_cloud_ml_sdk._types.operation:211] Fetching operation fvtk4abrkk7fsc772lq6 status
[D 2025-04-02 04:20:42 yandex_cloud_ml_sdk._types.operation:225] Operation fvtk4abrkk7fsc772lq6 have status OperationStatus<success>
[I 2025-04-02 04:20:42 yandex_cloud_ml_sdk._types.operation:118] Operation fvtk4abrkk7fsc772lq6 successfully finished
[I 2025-04-02 04:20:42 yandex_cloud_ml_sdk._types.operation:142]

Посмотрим, стал ли ответ системы лучше:

In [26]:
thread = create_thread()

thread.write("Какое вино подходит к стейку?")
run = assistant.run(thread)

result = run.wait()
print(result.text)
print_citations(result)

[I 2025-04-02 04:20:49 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvtt7v272897fueu81jv polling with a poll interval 0.500000s and poll timeout 300.000000s
[D 2025-04-02 04:20:49 yandex_cloud_ml_sdk._types.operation:110] Operation fvtt7v272897fueu81jv have non-terminal status PENDING, sleep for 0.500000s
[D 2025-04-02 04:20:50 yandex_cloud_ml_sdk._types.operation:110] Operation fvtt7v272897fueu81jv have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 04:20:50 yandex_cloud_ml_sdk._types.operation:110] Operation fvtt7v272897fueu81jv have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 04:20:51 yandex_cloud_ml_sdk._types.operation:110] Operation fvtt7v272897fueu81jv have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 04:20:51 yandex_cloud_ml_sdk._types.operation:110] Operation fvtt7v272897fueu81jv have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 04:20:52 yandex_cloud_ml_sdk._types.operation:1

К стейку из говядины подойдут следующие вина:

* Для нежного мраморного стейка (Филе-миньон) рекомендуются лёгкие и элегантные красные вина из винограда Пино Нуар, Нерелло Маскалезе, а также выдержанное и элегантное Мерло. Также подойдут «округлые», выдержанные варианты из сортов Неббиоло (Барбареско), Темпранильо (Рибейра дель Дуэро), Санджовезе (Кьянти Ризерва).

* Для жирноватого стейка (Рибай и подобные) с прожаркой Rare рекомендуются выдержанные и «благородные» вина из Темпранильо (Рибейра дель Дуэро или любые от Ризервы и выше), Санджовезе (Кьянти Ризерва, Брунелло), «супертосканские» вина, Бордо Правого берега, шелковистые аргентинские Мальбеки. С прожаркой Medium или WellDone хорошо сочетаются сухие и полусухие из винограда Сира (Шираз), Каберне Совиньон, «тельный» Мальбек, Примитиво, Зинфандель, Альянико (выдержанное и слегка «округлившееся»), выдержанный «ронский» ассамбляж Гренаш+Сира+Мурведр, вина Приората от 6–8 лет выдержки и выше.

Если речь идёт о стейке из лосося, то к

In [27]:
thread.delete()
assistant.delete()

## Function Calling

Предположим, мы делаем ассистента для магазина вин, который торгует винами по некоторому прайс-листу. Возьмём пример такого прайс-листа:

In [28]:
import pandas as pd

pl = pd.read_excel("source/wine-price.xlsx")
pl

,Номер артикула,название,CT,цена от 1 бутылки,от 3-х и более,регулярная цена/промо
0,56885,"0,75Л ВИНО САССИКАЙЯ КР СХ",IT,27799.000,19459.3000,NaN
1,666560,"0,75ВИНО СИЕПИ МАЗЕЙ КР СХ",IT,15999.000,11199.3000,NaN
2,533769,"0,75ВИНО ПАЛАФРЕНО КР СХ",IT,14999.004,10499.3028,NaN
3,93733,"0,75ВИНО АНТ ТИНЬЯНЕЛЛО КР СХ",IT,14499.012,10149.3084,NaN
4,644863,"0,75ВИНО ШАТО МОНРОЗ КР СХ",FR,12999.000,9099.3000,от промо цены
...,...,...,...,...,...,...
747,61418,"0,7ВИНО КАГОР ТАМ КР СЛ",RU,179.004,125.3028,от промо цены
748,615581,"0,187ВИНО ДЖАСТ МЕРЛО КР СХ",FR,149.004,104.3028,NaN
749,615582,"0,187ВИНО ДЖАСТ КБСВ КР СХ",FR,149.004,104.3028,NaN
750,83302,"0,187Л ВИНО АДАГУМ КБСВ КР СХ",RU,119.004,83.3028,NaN


Для удобства переименуем колонки:

In [29]:
pl.columns = ["Id", "Name", "Country", "Price", "WHPrice", "etc"]

Из имени извлечём кислотность вина:

In [30]:
acid_map = {"СХ": "Сухое", "СЛ": "Сладкое", "ПСХ": "Полусухое", "ПСЛ": "Полусладкое"}
pl["Acidity"] = pl["Name"].apply(
    lambda x: acid_map.get(x.split()[-1].replace("КР", ""), "")
)
pl["Acidity"].value_counts()

Сухое          614
Полусухое       68
Полусладкое     28
Сладкое         23
                19
Name: Acidity, dtype: int64

Тоже самое сделаем с цветом:

In [31]:
pl["Color"] = pl["Name"].apply(
    lambda x: (
        "Красное" if (x.split()[-1].startswith("КР") or x.split()[-2] == "КР") else ""
    )
)
pl["Color"].value_counts()

Красное    739
            13
Name: Color, dtype: int64

В итоге получилась такая таблица вин:

In [32]:
pl

,Id,Name,Country,Price,WHPrice,etc,Acidity,Color
0,56885,"0,75Л ВИНО САССИКАЙЯ КР СХ",IT,27799.000,19459.3000,NaN,Сухое,Красное
1,666560,"0,75ВИНО СИЕПИ МАЗЕЙ КР СХ",IT,15999.000,11199.3000,NaN,Сухое,Красное
2,533769,"0,75ВИНО ПАЛАФРЕНО КР СХ",IT,14999.004,10499.3028,NaN,Сухое,Красное
3,93733,"0,75ВИНО АНТ ТИНЬЯНЕЛЛО КР СХ",IT,14499.012,10149.3084,NaN,Сухое,Красное
4,644863,"0,75ВИНО ШАТО МОНРОЗ КР СХ",FR,12999.000,9099.3000,от промо цены,Сухое,Красное
...,...,...,...,...,...,...,...,...
747,61418,"0,7ВИНО КАГОР ТАМ КР СЛ",RU,179.004,125.3028,от промо цены,Сладкое,Красное
748,615581,"0,187ВИНО ДЖАСТ МЕРЛО КР СХ",FR,149.004,104.3028,NaN,Сухое,Красное
749,615582,"0,187ВИНО ДЖАСТ КБСВ КР СХ",FR,149.004,104.3028,NaN,Сухое,Красное
750,83302,"0,187Л ВИНО АДАГУМ КБСВ КР СХ",RU,119.004,83.3028,NaN,Сухое,Красное


Чтобы научиться отвечать на вопросы по прайс-листу, у нас есть несколько возможных решений:

* Попытаться закинуть прайс-лист в контекст модели с помощью RAG, как мы это ранее делали с таблицей блюд. Не очень хороший вариант, потому что модель не будет видеть всю таблицу, и не сможет отвечать на вопросы "максимальная/минимальная цена".
* Попытаться организовать трансляцию запроса не естественном языке в SQL-подобный язык. Это идеальный вариант, но его сложно сделать без ошибок без fine-tuning-а модели. 
* Извлечь из текстового запроса основные параметры того, что хочет пользователь, и затем сформировать на этой основе запрос, извлечающий данные из таблицы. Такой подход описан, например, в статье [Querying Databases with Function Calling](https://arxiv.org/html/2502.00032v1)

В любом случае, два последних варианта подразумевают использование технологии Function Calling - когда модель может сама решить, когда ей надо вызвать некоторую **внешнюю функцию**, и какие параметры нужно ей передать.

Чтобы function calling работал - нам надо сообщить LLM о доступных **инструментах**. Это можно сделать, передав с помощью JSON-схемы описание возможностей таких инструментов и их параметров.

ML SDK позволяет нам упростить function calling, и вместо JSON-схемы использовать типизированные объекты Pyton. Для извлечения параметров запроса о вине, мы создадим такой объект: 

In [33]:
from pydantic import BaseModel, Field
from typing import Optional

class SearchWinePriceList(BaseModel):
    """Эта функция позволяет искать вина в прайс-листе по одному или нескольким параметрам."""

    name: str = Field(description="Название вина", default=None)
    country: str = Field(description="Страна", default=None)
    acidity: str = Field(
        description="Кислотность (сухое, полусухое, сладкое, полусладкое)", default=None
    )
    color: str = Field(description="Цвет вина (красное, белое, розовое)", default=None)
    sort_order: str = Field(
        description="Порядок выдачи (most expensive, cheapest, random, average)",
        default=None,
    )
    what_to_return: str = Field(
        description="Что вернуть (wine info или price)", default=None
    )

Теперь создадим инструмент (tool) и нового ассистента, у которого в списке инструментов будет одновременно и RAG-поиск, и function calling. Также в инструкции ассистенту пропишем, что он может использовать Function Calling.

In [37]:
price_list_search_tool = sdk.tools.function(SearchWinePriceList)

assistant = create_assistant(model, tools=[price_list_search_tool, search_tool])
thread = create_thread()

instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина. 
Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций. Если вопрос касается конкретных вин
или цены, то используй Function Calling.
Если что-то непонятно, то лучше уточни информацию у пользователя.
"""

_ = assistant.update(instruction=instruction)

[D 2025-04-02 04:23:13 yandex_cloud_ml_sdk._types.schemas:98] trasform input parameters=<class '__main__.SearchWinePriceList'> to json_schema={'description': 'Эта функция позволяет искать вина в прайс-листе по одному или нескольким параметрам.', 'properties': {'name': {'default': None, 'description': 'Название вина', 'title': 'Name', 'type': 'string'}, 'country': {'default': None, 'description': 'Страна', 'title': 'Country', 'type': 'string'}, 'acidity': {'default': None, 'description': 'Кислотность (сухое, полусухое, сладкое, полусладкое)', 'title': 'Acidity', 'type': 'string'}, 'color': {'default': None, 'description': 'Цвет вина (красное, белое, розовое)', 'title': 'Color', 'type': 'string'}, 'sort_order': {'default': None, 'description': 'Порядок выдачи (most expensive, cheapest, random, average)', 'title': 'Sort Order', 'type': 'string'}, 'what_to_return': {'default': None, 'description': 'Что вернуть (wine info или price)', 'title': 'What To Return', 'type': 'string'}}, 'title': 

Попробуем узнать самое дешевое вино из Австралии:

In [41]:
thread.write("Привет! Какое самое дешевое красное сухое вино из Австралии?")
run = assistant.run(thread)
res = run.wait()
res

[I 2025-04-02 04:24:18 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvtqa08e0qvq22rmm2oc polling with a poll interval 0.500000s and poll timeout 300.000000s
[D 2025-04-02 04:24:18 yandex_cloud_ml_sdk._types.operation:110] Operation fvtqa08e0qvq22rmm2oc have non-terminal status PENDING, sleep for 0.500000s
[D 2025-04-02 04:24:18 yandex_cloud_ml_sdk._types.operation:110] Operation fvtqa08e0qvq22rmm2oc have non-terminal status IN_PROGRESS, sleep for 0.500000s
[I 2025-04-02 04:24:19 yandex_cloud_ml_sdk._types.operation:118] Operation fvtqa08e0qvq22rmm2oc successfully finished
[I 2025-04-02 04:24:19 yandex_cloud_ml_sdk._types.operation:142] Operation fvtqa08e0qvq22rmm2oc polling finished


RunResult(status=<RunStatus.TOOL_CALLS: 5>, error=None, tool_calls=ToolCallList(ToolCall(function=FunctionCall(name='SearchWinePriceList', arguments={'what_to_return': 'price', 'acidity': 'сухое', 'color': 'красное', 'sort_order': 'cheapest', 'country': 'Австралия'})),), _message=None, usage=Usage(input_text_tokens=2693, completion_tokens=39, total_tokens=2732))

Видим, что в итоге вернулся ответ, указывающий, что надо сделать Funtion Call для получения данных из таблицы. При это LLM извлекла из текста все параметры запроса.

Реализуем функцию, которая возвращает список вин по параметрам, заданным в виде объекта `SearchWinePriceList`:

In [39]:
country_map = {
    "IT": "Италия",
    "FR": "Франция",
    "ES": "Испания",
    "RU": "Россия",
    "PT": "Португалия",
    "AR": "Армения",
    "CL": "Чили",
    "AU": "Австрия",
    "GE": "Грузия",
    "ZA": "ЮАР",
    "US": "США",
    "NZ": "Новая Зеландия",
    "DE": "Германия",
    "AT": "Австрия",
    "IL": "Израиль",
    "BG": "Болгария",
    "GR": "Греция",
    "AU": "Австралия",
}

revmap = {v.lower(): k for k, v in country_map.items()}


def find_wines(req):
    x = pl.copy()
    if req.country and req.country.lower() in revmap.keys():
        x = x[x["Country"] == revmap[req.country.lower()]]
    if req.acidity:
        x = x[x["Acidity"] == req.acidity.capitalize()]
    if req.color:
        x = x[x["Color"] == req.color.capitalize()]
    if req.name:
        x = x[x["Name"].apply(lambda x: req.name.lower() in x.lower())]
    if req.sort_order:
        if req.sort_order == "cheapest":
            x = x.sort_values(by="Price")
        elif req.sort_order == "most expensive":
            x = x.sort_values(by="Price", ascending=False)
        else:
            pass
    if x is None or len(x) == 0:
        return "Подходящих вин не найдено"
    return "Вот какие вина были найдены:\n" + "\n".join(
        [
            f"{z['Name']} ({country_map.get(z['Country'],'Неизвестно')}) - {z['Price']}"
            for _, z in x.head(10).iterrows()
        ]
    )


print(find_wines(SearchWinePriceList(country="Австралия", sort_order="cheapest")))

Вот какие вина были найдены:
0,75ВИНО ДЖИНДАЛИ МЕРЛО КР ПСХ (Австралия) - 499.0
0,75ВИНО ДЖИНДАЛИ КБСВ КР ПСХ (Австралия) - 499.0
0,75ВИНО ЧОЛК ХИЛЛ ШИРАЗ КР СХ (Австралия) - 509.0
0,75ВИНО ПИТ'С ПЮР ШИРАЗ КР ПСХ (Австралия) - 579.0
0,75ВИНО ПИТ'С ПЮР ПННР КР ПСХ (Австралия) - 579.0
0,75ВИНО СТАМП ДЖАМП КР СХ (Австралия) - 789.0
0,75ВИНО ЛИНД БИН50 ШИР КР ПСХ (Австралия) - 899.0
0,75ВИНО ЛЭКИ ШИРАЗ КРСХ (Австралия) - 978.996
0,75ВИНО СТЭДФАСТ ШИР КАБ КРСХ (Австралия) - 999.0
0,75ВИНО ТИРРЕЛЗ ШИР КР СХ (Австралия) - 1098.996


В ответ на Function Call нам нужно сформировать ответ от функции, и передать её обратно для обработки языковой моделью. В нашем случае функция может быть только одна, поэтому мы не проверяем название функции, а всегда запрашиваем поиск в таблице:

In [42]:
import time

if res.tool_calls:
    result = []
    for f in res.tool_calls:
        print(f" + Processing function call fn={f.function.name}")
        x = SearchWinePriceList.model_validate(f.function.arguments)
        x = find_wines(x)
        result.append({"name": f.function.name, "content": x})
    run.submit_tool_results(result)
    time.sleep(3)
    res = run.wait()
res

 + Processing function call fn=SearchWinePriceList


[I 2025-04-02 04:24:27 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvtqa08e0qvq22rmm2oc polling with a poll interval 0.500000s and poll timeout 300.000000s
[I 2025-04-02 04:24:27 yandex_cloud_ml_sdk._types.operation:118] Operation fvtqa08e0qvq22rmm2oc successfully finished
[I 2025-04-02 04:24:27 yandex_cloud_ml_sdk._types.operation:142] Operation fvtqa08e0qvq22rmm2oc polling finished


RunResult(status=<RunStatus.COMPLETED: 4>, error=None, tool_calls=None, _message=Message(id='fvts2dqmlmvdu43mc4f6', parts=('Самое дешёвое красное сухое вино из Австралии в нашем магазине — это «Чолк Хилл Шираз», стоимостью 509 рублей за бутылку объёмом 0,75 литра.',), thread_id='fvt01m53ghe08fpnhdta', created_by='ajej20rll4tifkelclga', created_at=datetime.datetime(2025, 4, 2, 4, 24, 24, 768989), labels=None, author=Author(id='fvtg2fi531uihqlcf7mr', role='ASSISTANT'), citations=()), usage=Usage(input_text_tokens=3385, completion_tokens=77, total_tokens=3462))

In [43]:
thread.delete()
assistant.delete()

## Релизуем агента с Function Calling

Для упрощения реализации Function Calling напишем небольшую обвязку, реализующую агента, способного искать в текствой базе и делать Function Calling. 

Функцию обработки запроса мы включим в состав класса для описания функции, назовём её `process`. Для реализации всех наших задумок также будем передавать в неё текущий `thread`:

In [44]:
class SearchWinePriceList(BaseModel):
    """Эта функция позволяет искать вина в прайс-листе по одному или нескольким параметрам."""

    name: str = Field(description="Название вина", default=None)
    country: str = Field(description="Страна", default=None)
    acidity: str = Field(
        description="Кислотность (сухое, полусухое, сладкое, полусладкое)", default=None
    )
    color: str = Field(description="Цвет вина (красное, белое, розовое)", default=None)
    sort_order: str = Field(
        description="Порядок выдачи (most expensive, cheapest, random, average)",
        default=None,
    )
    what_to_return: str = Field(
        description="Что вернуть (wine info или price)", default=None
    )

    def process(self, thread):
        return find_wines(self)

Также в виде функции реализуем функциональность передачи управления оператору. В данном случае мы будем просто устанавливать некоторый глобальный флаг `handover` - это очень грубое решение и сделано для простоты, в реальной жизни надо будет подумать над более правильной логикой, поддерживающей нескольких пользователей.

In [46]:
handover = False

class Handover(BaseModel):
    """Эта функция позволяет передать диалог человеку-оператору поддержки"""

    reason: str = Field(
        description="Причина для вызова оператора", default="не указана"
    )

    def process(self, thread):
        global handover
        handover = True
        return f"Я побежала вызывать оператора, ваш {thread.id=}, причина: {self.reason}"

Также реализуем функцию добавления вин в корзину. Чтобы для каждого пользователя была своя корзина, будем привязывать её к `thread.id` с помощью словаря `carts`:

In [47]:
carts = {}


class AddToCart(BaseModel):
    """Эта функция позволяет положить вино в корзину"""

    wine_name: str = Field(
        description="Точное название вина, чтобы положить в корзину", default=None
    )
    count: str = Field(
        description="Количество бутылок вина, которое нужно положить в корзину",
        default=1,
    )

    def process(self, thread):
        if thread.id not in carts:
            carts[thread.id] = []
        carts[thread.id].append(self)
        return f"Вино {self.wine_name} добавлено в корзину, число бутылок: {self.count}"

Наконец, оформим функцию для показа корзины:

In [62]:
class ShowCart(BaseModel):
    """Эта функция позволяет показать содержимое корзины"""

    def process(self, thread):
        if thread.id not in carts or len(carts[thread.id]) == 0:
            return "Корзина пуста"
        return "В корзине находятся следующие вина:\n" + "\n".join(
            [f"{x.wine_name}, число бутылок: {x.count}" for x in carts[thread.id]]
        )

Теперь реализуем главный класс `Agent`, который будет брать на себя обработку функций. В качестве `tools` будем передавать список описанных нами ранее классов.

Также предусмотрим гибкую работу с `thread`. При запросе агента мы сможем опиционально указывать ему уже созданный `thread` для ведения переписки, либо же переписка будет вестись в созданном потоке по умолчанию. Это полезно для упрощения нашего дальнейшего кода.

In [63]:
class Agent:
    def __init__(self, assistant=None, instruction=None, search_index=None, tools=None):

        self.thread = None

        if assistant:
            self.assistant = assistant
        else:
            if tools:
                self.tools = {x.__name__: x for x in tools}
                tools = [sdk.tools.function(x) for x in tools]
            else:
                self.tools = {}
                tools = []
            if search_index:
                tools.append(sdk.tools.search_index(search_index))
            self.assistant = create_assistant(model, tools)

        if instruction:
            self.assistant.update(instruction=instruction)

    def get_thread(self, thread=None):
        if thread is not None:
            return thread
        if self.thread == None:
            self.thread = create_thread()
        return self.thread

    def __call__(self, message, thread=None):
        thread = self.get_thread(thread)
        thread.write(message)
        run = self.assistant.run(thread)
        res = run.wait()
        if res.tool_calls:
            result = []
            for f in res.tool_calls:
                print(
                    f" + Вызываем функцию {f.function.name}, args={f.function.arguments}"
                )
                fn = self.tools[f.function.name]
                obj = fn(**f.function.arguments)
                x = obj.process(thread)
                result.append({"name": f.function.name, "content": x})
            run.submit_tool_results(result)
            #time.sleep(3)
            res = run.wait()
        return res.text

    def restart(self):
        if self.thread:
            self.thread.delete()
            self.thread = sdk.threads.create(
                name="Test", ttl_days=1, expiration_policy="static"
            )

    def done(self, delete_assistant=False):
        if self.thread:
            self.thread.delete()
        if delete_assistant:
            self.assistant.delete()

Создадим нашего агента - винного сомелье. В системном промпте пропишем ему возможные варианты для вызова функции, чтобы облегчить задачу.

In [64]:
instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина. 
Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций.
Если вопрос касается конкретных вин или цены, то вызови функцию SearchWinePriceList.
Для передачи управления оператору - вызови фукцию Handover. Для добавления вина в корзину
используй AddToCart. Для просмотра корзины: ShowCart.
Если что-то непонятно, то лучше уточни информацию у пользователя.
"""

wine_agent = Agent(
    instruction=instruction,
    search_index=index,
    tools=[SearchWinePriceList, Handover, AddToCart, ShowCart],
)

[D 2025-04-02 04:45:30 yandex_cloud_ml_sdk._types.schemas:98] trasform input parameters=<class '__main__.SearchWinePriceList'> to json_schema={'description': 'Эта функция позволяет искать вина в прайс-листе по одному или нескольким параметрам.', 'properties': {'name': {'default': None, 'description': 'Название вина', 'title': 'Name', 'type': 'string'}, 'country': {'default': None, 'description': 'Страна', 'title': 'Country', 'type': 'string'}, 'acidity': {'default': None, 'description': 'Кислотность (сухое, полусухое, сладкое, полусладкое)', 'title': 'Acidity', 'type': 'string'}, 'color': {'default': None, 'description': 'Цвет вина (красное, белое, розовое)', 'title': 'Color', 'type': 'string'}, 'sort_order': {'default': None, 'description': 'Порядок выдачи (most expensive, cheapest, random, average)', 'title': 'Sort Order', 'type': 'string'}, 'what_to_return': {'default': None, 'description': 'Что вернуть (wine info или price)', 'title': 'What To Return', 'type': 'string'}}, 'title': 

In [65]:
print(wine_agent("Какое вино пьют со стейком?"))

[I 2025-04-02 04:45:31 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvt3rvrcn6u7h4f0ttl8 polling with a poll interval 0.500000s and poll timeout 300.000000s
[D 2025-04-02 04:45:31 yandex_cloud_ml_sdk._types.operation:110] Operation fvt3rvrcn6u7h4f0ttl8 have non-terminal status PENDING, sleep for 0.500000s
[D 2025-04-02 04:45:32 yandex_cloud_ml_sdk._types.operation:110] Operation fvt3rvrcn6u7h4f0ttl8 have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 04:45:32 yandex_cloud_ml_sdk._types.operation:110] Operation fvt3rvrcn6u7h4f0ttl8 have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 04:45:33 yandex_cloud_ml_sdk._types.operation:110] Operation fvt3rvrcn6u7h4f0ttl8 have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 04:45:33 yandex_cloud_ml_sdk._types.operation:110] Operation fvt3rvrcn6u7h4f0ttl8 have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 04:45:34 yandex_cloud_ml_sdk._types.operation:1

Для стейка можно порекомендовать следующие вина:

1. Для нежного мраморного стейка (Филе-миньон) подойдут лёгкие и элегантные красные вина из винограда Пино Нуар, Нерелло Маскалезе, а также выдержанное и элегантное Мерло. Кроме того, можно рассмотреть «округлые», выдержанные варианты из сортов Неббиоло (Барбареско), Темпранильо (Рибейра дель Дуэро), Санджовезе (Кьянти Ризерва).

2. Для мраморного жирноватого стейка (Рибай и пр.) в зависимости от прожарки можно выбрать:
   - Для прожарки Rare — выдержанные и «благородные» вина из Темпранильо (Рибейра дель Дуэро или любые от Ризервы и выше), Санджовезе (Кьянти Ризерва, Брунелло), «супертосканские» вина, Бордо Правого берега, шелковистые аргентинские Мальбеки.
   - Для прожарки Medium или WellDone — сухие и полусухие из винограда Сира (Шираз), Каберне Совиньон, «тельный» Мальбек, Примитиво, Зинфандель, Альянико (выдержанное и слегка «округлившееся»), выдержанный «ронский» ассамбляж Гренаш+Сира+Мурведр, вина Приората от 6–8 лет выдержки и 

In [66]:
print(wine_agent("Какие вина Кьянти есть в продаже?"))

[I 2025-04-02 04:45:41 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvtli5rbgqieiaan339g polling with a poll interval 0.500000s and poll timeout 300.000000s
[D 2025-04-02 04:45:41 yandex_cloud_ml_sdk._types.operation:110] Operation fvtli5rbgqieiaan339g have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 04:45:41 yandex_cloud_ml_sdk._types.operation:110] Operation fvtli5rbgqieiaan339g have non-terminal status IN_PROGRESS, sleep for 0.500000s
[I 2025-04-02 04:45:42 yandex_cloud_ml_sdk._types.operation:118] Operation fvtli5rbgqieiaan339g successfully finished
[I 2025-04-02 04:45:42 yandex_cloud_ml_sdk._types.operation:142] Operation fvtli5rbgqieiaan339g polling finished


 + Вызываем функцию SearchWinePriceList, args={'what_to_return': 'wine info', 'name': 'Кьянти'}


[I 2025-04-02 04:45:46 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvtli5rbgqieiaan339g polling with a poll interval 0.500000s and poll timeout 300.000000s
[I 2025-04-02 04:45:46 yandex_cloud_ml_sdk._types.operation:118] Operation fvtli5rbgqieiaan339g successfully finished
[I 2025-04-02 04:45:46 yandex_cloud_ml_sdk._types.operation:142] Operation fvtli5rbgqieiaan339g polling finished


В нашем магазине в продаже есть следующие вина Кьянти:

1. Вино Кверчаб Кьянти красное сухое, 0,75 л - 2499.0 руб.
2. Вино Полиц Кьянти красное сухое, 0,75 л - 1749.756 руб.
3. Вино Касал Кьянти супер красное сухое, 0,75 л - 1349.004 руб.
4. Вино Век Кант Кьянти красное сухое, 0,75 л - 1099.0 руб.
5. Вино Пределла Кьянти красное сухое, 1,5 л - 999.0 руб.
6. Вино Зонин Кьянти красное сухое, 0,75 л - 699.0 руб.
7. Вино Пределла Кьянти красное сухое, 0,75 л - 369.0 руб.


In [67]:
print(wine_agent("Добавь в корзину Полиц Кьянти, три бутылки"))

[I 2025-04-02 04:45:48 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvtkq239aj0970s6f14q polling with a poll interval 0.500000s and poll timeout 300.000000s
[D 2025-04-02 04:45:48 yandex_cloud_ml_sdk._types.operation:110] Operation fvtkq239aj0970s6f14q have non-terminal status PENDING, sleep for 0.500000s
[D 2025-04-02 04:45:49 yandex_cloud_ml_sdk._types.operation:110] Operation fvtkq239aj0970s6f14q have non-terminal status IN_PROGRESS, sleep for 0.500000s
[I 2025-04-02 04:45:49 yandex_cloud_ml_sdk._types.operation:118] Operation fvtkq239aj0970s6f14q successfully finished
[I 2025-04-02 04:45:49 yandex_cloud_ml_sdk._types.operation:142] Operation fvtkq239aj0970s6f14q polling finished


 + Вызываем функцию AddToCart, args={'wine_name': 'Полиц Кьянти', 'count': '3'}


[I 2025-04-02 04:45:50 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvtkq239aj0970s6f14q polling with a poll interval 0.500000s and poll timeout 300.000000s
[I 2025-04-02 04:45:50 yandex_cloud_ml_sdk._types.operation:118] Operation fvtkq239aj0970s6f14q successfully finished
[I 2025-04-02 04:45:50 yandex_cloud_ml_sdk._types.operation:142] Operation fvtkq239aj0970s6f14q polling finished


Вино Полиц Кьянти успешно добавлено в корзину в количестве трёх бутылок. Если вам нужно что-то ещё или хотите просмотреть содержимое корзины, дайте знать!


In [68]:
print(wine_agent("Ещё добавь в корзину Зонин Кьянти"))

[I 2025-04-02 04:45:53 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvtfv1au6qa8mt48khc5 polling with a poll interval 0.500000s and poll timeout 300.000000s
[D 2025-04-02 04:45:53 yandex_cloud_ml_sdk._types.operation:110] Operation fvtfv1au6qa8mt48khc5 have non-terminal status PENDING, sleep for 0.500000s
[D 2025-04-02 04:45:54 yandex_cloud_ml_sdk._types.operation:110] Operation fvtfv1au6qa8mt48khc5 have non-terminal status IN_PROGRESS, sleep for 0.500000s
[I 2025-04-02 04:45:54 yandex_cloud_ml_sdk._types.operation:118] Operation fvtfv1au6qa8mt48khc5 successfully finished
[I 2025-04-02 04:45:54 yandex_cloud_ml_sdk._types.operation:142] Operation fvtfv1au6qa8mt48khc5 polling finished


 + Вызываем функцию AddToCart, args={'wine_name': 'Зонин Кьянти', 'count': '1'}


[I 2025-04-02 04:45:55 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvtfv1au6qa8mt48khc5 polling with a poll interval 0.500000s and poll timeout 300.000000s
[I 2025-04-02 04:45:55 yandex_cloud_ml_sdk._types.operation:118] Operation fvtfv1au6qa8mt48khc5 successfully finished
[I 2025-04-02 04:45:55 yandex_cloud_ml_sdk._types.operation:142] Operation fvtfv1au6qa8mt48khc5 polling finished


Вино Зонин Кьянти успешно добавлено в корзину. Если вам нужно что-то ещё или хотите просмотреть содержимое корзины, сообщите мне!


In [69]:
print(wine_agent("Что у меня в корзине?"))

[I 2025-04-02 04:45:59 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvtbmk2jm30bjitbi3qg polling with a poll interval 0.500000s and poll timeout 300.000000s
[D 2025-04-02 04:45:59 yandex_cloud_ml_sdk._types.operation:110] Operation fvtbmk2jm30bjitbi3qg have non-terminal status PENDING, sleep for 0.500000s
[D 2025-04-02 04:46:00 yandex_cloud_ml_sdk._types.operation:110] Operation fvtbmk2jm30bjitbi3qg have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 04:46:00 yandex_cloud_ml_sdk._types.operation:110] Operation fvtbmk2jm30bjitbi3qg have non-terminal status IN_PROGRESS, sleep for 0.500000s
[I 2025-04-02 04:46:01 yandex_cloud_ml_sdk._types.operation:118] Operation fvtbmk2jm30bjitbi3qg successfully finished
[I 2025-04-02 04:46:01 yandex_cloud_ml_sdk._types.operation:142] Operation fvtbmk2jm30bjitbi3qg polling finished


 + Вызываем функцию ShowCart, args={}


[I 2025-04-02 04:46:01 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvtbmk2jm30bjitbi3qg polling with a poll interval 0.500000s and poll timeout 300.000000s
[I 2025-04-02 04:46:01 yandex_cloud_ml_sdk._types.operation:118] Operation fvtbmk2jm30bjitbi3qg successfully finished
[I 2025-04-02 04:46:01 yandex_cloud_ml_sdk._types.operation:142] Operation fvtbmk2jm30bjitbi3qg polling finished


В вашей корзине находятся следующие вина:
- Полиц Кьянти, количество бутылок: 3
- Зонин Кьянти, количество бутылок: 1


In [70]:
print(wine_agent("Вызови оператора, хочу оформить доставку!"))

[I 2025-04-02 04:46:06 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvtpfsv85i1eqchk75fk polling with a poll interval 0.500000s and poll timeout 300.000000s
[D 2025-04-02 04:46:06 yandex_cloud_ml_sdk._types.operation:110] Operation fvtpfsv85i1eqchk75fk have non-terminal status PENDING, sleep for 0.500000s
[D 2025-04-02 04:46:06 yandex_cloud_ml_sdk._types.operation:110] Operation fvtpfsv85i1eqchk75fk have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 04:46:07 yandex_cloud_ml_sdk._types.operation:110] Operation fvtpfsv85i1eqchk75fk have non-terminal status IN_PROGRESS, sleep for 0.500000s
[I 2025-04-02 04:46:07 yandex_cloud_ml_sdk._types.operation:118] Operation fvtpfsv85i1eqchk75fk successfully finished
[I 2025-04-02 04:46:07 yandex_cloud_ml_sdk._types.operation:142] Operation fvtpfsv85i1eqchk75fk polling finished


 + Вызываем функцию Handover, args={'reason': 'Пользователь хочет оформить доставку'}


[I 2025-04-02 04:46:10 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvtpfsv85i1eqchk75fk polling with a poll interval 0.500000s and poll timeout 300.000000s
[I 2025-04-02 04:46:10 yandex_cloud_ml_sdk._types.operation:118] Operation fvtpfsv85i1eqchk75fk successfully finished
[I 2025-04-02 04:46:10 yandex_cloud_ml_sdk._types.operation:142] Operation fvtpfsv85i1eqchk75fk polling finished


Ожидайте, скоро с вами свяжется оператор для оформления доставки.


## Многоагентное тестирование

Когда мы сделали такого бота, возникает вопрос, как его тестировать. Для этого возможно несколько вариантов:

* Ручное тестирование (примерно то, что мы проделали выше)
* Автоматическое тестирование на заранее заготовленном датасете диалогов, с формализованной проверкой метрик. Такое тестирование удобно проводить с помощью специализированных фреймворков, например, RAGAS.

Мы в данном примере предлагаем промежуточный по сложности подход - это **многоагентное тестирование**, при котором мы инструктируем другого агента, чтобы он побеседовал с нашим винным ассистентом в определённом ключе. В итоге у нас могут получиться различные сценарии диалога, глядя на которые можно будет определить направление доработки исходного ассистента.

In [73]:
instruction_user = """
Ты - простой программист, и тебе нужно выбрать вино для ужина в интернет-магазине.
Ты не очень много понимаешь в винах, и хочешь побольше распросить сомелье о разных вариантах.
Ты хотел бы съесть салат капрезе и стейк, но готов изменить свой выбор еды, если поймёшь хорошее сочетание с вином. Говори
простым языком, короткими фразами. Когда ты удовлетворём выбором, попроси найти самое недорогое
вино данного типа, затем положи его в корзину и попроси соединить тебя с оператором, чтобы
оформить доставку. Каждое действие выполняй в одной фразой в ответ на сообщение сомелье.
"""

user = Agent(instruction=instruction_user)

msg = "Добрый день! Поможете мне выбрать вино?"
handover = False
for i in range(10):
    printx(f"**Посетитель:** {msg}")
    msg = wine_agent(msg)
    printx(f"**Сомелье:** {msg}")
    if handover:
        break
    msg = user(msg)

**Посетитель:** Добрый день! Поможете мне выбрать вино?

[I 2025-04-02 05:10:31 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvt8avk8fc4f7okab16a polling with a poll interval 0.500000s and poll timeout 300.000000s
[D 2025-04-02 05:10:31 yandex_cloud_ml_sdk._types.operation:110] Operation fvt8avk8fc4f7okab16a have non-terminal status PENDING, sleep for 0.500000s
[D 2025-04-02 05:10:32 yandex_cloud_ml_sdk._types.operation:110] Operation fvt8avk8fc4f7okab16a have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 05:10:32 yandex_cloud_ml_sdk._types.operation:110] Operation fvt8avk8fc4f7okab16a have non-terminal status IN_PROGRESS, sleep for 0.500000s
[I 2025-04-02 05:10:33 yandex_cloud_ml_sdk._types.operation:118] Operation fvt8avk8fc4f7okab16a successfully finished
[I 2025-04-02 05:10:33 yandex_cloud_ml_sdk._types.operation:142] Operation fvt8avk8fc4f7okab16a polling finished


**Сомелье:** Конечно, с удовольствием помогу! Для начала расскажите, пожалуйста, к какому блюду вы хотите подобрать вино?

[I 2025-04-02 05:10:33 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvto0gs83gqh9aj06di1 polling with a poll interval 0.500000s and poll timeout 300.000000s
[D 2025-04-02 05:10:33 yandex_cloud_ml_sdk._types.operation:110] Operation fvto0gs83gqh9aj06di1 have non-terminal status PENDING, sleep for 0.500000s
[I 2025-04-02 05:10:34 yandex_cloud_ml_sdk._types.operation:118] Operation fvto0gs83gqh9aj06di1 successfully finished
[I 2025-04-02 05:10:34 yandex_cloud_ml_sdk._types.operation:142] Operation fvto0gs83gqh9aj06di1 polling finished


**Посетитель:** Я хочу подобрать вино к капрезе и стейку.

...

**Посетитель:** «0,187 ВИНО ДЖАСТ МЕРЛО КР СХ (Франция)»

[I 2025-04-02 05:10:54 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvtudpis4ubckm7ibmjb polling with a poll interval 0.500000s and poll timeout 300.000000s
[D 2025-04-02 05:10:54 yandex_cloud_ml_sdk._types.operation:110] Operation fvtudpis4ubckm7ibmjb have non-terminal status PENDING, sleep for 0.500000s
[D 2025-04-02 05:10:55 yandex_cloud_ml_sdk._types.operation:110] Operation fvtudpis4ubckm7ibmjb have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 05:10:55 yandex_cloud_ml_sdk._types.operation:110] Operation fvtudpis4ubckm7ibmjb have non-terminal status IN_PROGRESS, sleep for 0.500000s
[I 2025-04-02 05:10:56 yandex_cloud_ml_sdk._types.operation:118] Operation fvtudpis4ubckm7ibmjb successfully finished
[I 2025-04-02 05:10:56 yandex_cloud_ml_sdk._types.operation:142] Operation fvtudpis4ubckm7ibmjb polling finished


 + Вызываем функцию AddToCart, args={'wine_name': '0,187 ВИНО ДЖАСТ МЕРЛО КР СХ (Франция)', 'count': '1'}
 + Вызываем функцию Handover, args={'reason': 'Оформить доставку'}


[I 2025-04-02 05:10:57 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvtudpis4ubckm7ibmjb polling with a poll interval 0.500000s and poll timeout 300.000000s
[I 2025-04-02 05:10:57 yandex_cloud_ml_sdk._types.operation:118] Operation fvtudpis4ubckm7ibmjb successfully finished
[I 2025-04-02 05:10:57 yandex_cloud_ml_sdk._types.operation:142] Operation fvtudpis4ubckm7ibmjb polling finished


**Сомелье:** Вино 0,187 ВИНО ДЖАСТ МЕРЛО КР СХ (Франция) успешно добавлено в корзину. Ожидайте, скоро с вами свяжется оператор для оформления доставки.

In [74]:
user.done(delete_assistant=True)
wine_agent.done(delete_assistant=True)

## Делаем ассистента проактивным

Мы обычно воспринимаем ассистентов как вопрос-ответных ботов, которые способны отвечать на вопросы пользователей. Однако с помощью системного промпта мы можем добавить им проактивности, чтобы они проявляли инициативу и сами инициировали диалог. Например:

In [75]:
instruction = """
Ты - опытный сомелье, продающий вино в магазине. Твоя задача - отвечать на вопросы пользователя
про вина, рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина,
а также проактивно предлагать пользователю приобрести вина, отвечающие его потребностям. В ответ
на сообщение /start поинтересуйся, что нужно пользователю, предложи ему какой-то
интересный вариант сочетания еды и вине, и попытайся продать ему вино.
Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций.
Если вопрос касается конкретных вин или цены, то вызови функцию SearchWinePriceList.
Для передачи управления оператору - вызови фукцию Handover. Для добавления вина в корзину
используй AddToCart. Для просмотра корзины: ShowCart.
Если что-то непонятно, то лучше уточни информацию у пользователя.
"""

wine_agent = Agent(
    instruction=instruction,
    search_index=index,
    tools=[SearchWinePriceList, Handover, AddToCart, ShowCart],
)

[D 2025-04-02 05:12:08 yandex_cloud_ml_sdk._types.schemas:98] trasform input parameters=<class '__main__.SearchWinePriceList'> to json_schema={'description': 'Эта функция позволяет искать вина в прайс-листе по одному или нескольким параметрам.', 'properties': {'name': {'default': None, 'description': 'Название вина', 'title': 'Name', 'type': 'string'}, 'country': {'default': None, 'description': 'Страна', 'title': 'Country', 'type': 'string'}, 'acidity': {'default': None, 'description': 'Кислотность (сухое, полусухое, сладкое, полусладкое)', 'title': 'Acidity', 'type': 'string'}, 'color': {'default': None, 'description': 'Цвет вина (красное, белое, розовое)', 'title': 'Color', 'type': 'string'}, 'sort_order': {'default': None, 'description': 'Порядок выдачи (most expensive, cheapest, random, average)', 'title': 'Sort Order', 'type': 'string'}, 'what_to_return': {'default': None, 'description': 'Что вернуть (wine info или price)', 'title': 'What To Return', 'type': 'string'}}, 'title': 

In [76]:
printx(wine_agent('/start'))

[I 2025-04-02 05:12:13 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvtb8s5dnoue9aqecp5j polling with a poll interval 0.500000s and poll timeout 300.000000s
[D 2025-04-02 05:12:13 yandex_cloud_ml_sdk._types.operation:110] Operation fvtb8s5dnoue9aqecp5j have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 05:12:14 yandex_cloud_ml_sdk._types.operation:110] Operation fvtb8s5dnoue9aqecp5j have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 05:12:14 yandex_cloud_ml_sdk._types.operation:110] Operation fvtb8s5dnoue9aqecp5j have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 05:12:15 yandex_cloud_ml_sdk._types.operation:110] Operation fvtb8s5dnoue9aqecp5j have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 05:12:15 yandex_cloud_ml_sdk._types.operation:110] Operation fvtb8s5dnoue9aqecp5j have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 05:12:16 yandex_cloud_ml_sdk._types.operati

Здравствуйте! Рад видеть вас в нашем винном магазине. Чем могу помочь?

Возможно, вас заинтересует сочетание тушеной спаржи с белым сухим вином из региона Элим в ЮАР. Это вино обладает насыщенным вкусом и отлично дополнит ваше блюдо.

Или, например, каберне совиньон из Калифорнии, который прекрасно гармонирует с пеппер-стейком.

Какое вино вы предпочитаете? Или, может быть, у вас есть определенное блюдо, к которому вы хотите подобрать вино?

In [77]:
printx(wine_agent('Хочу вино к стейку'))

[I 2025-04-02 05:12:22 yandex_cloud_ml_sdk._types.operation:131] Starting operation fvtcuk5pj2erssq21335 polling with a poll interval 0.500000s and poll timeout 300.000000s
[D 2025-04-02 05:12:22 yandex_cloud_ml_sdk._types.operation:110] Operation fvtcuk5pj2erssq21335 have non-terminal status PENDING, sleep for 0.500000s
[D 2025-04-02 05:12:23 yandex_cloud_ml_sdk._types.operation:110] Operation fvtcuk5pj2erssq21335 have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 05:12:23 yandex_cloud_ml_sdk._types.operation:110] Operation fvtcuk5pj2erssq21335 have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 05:12:24 yandex_cloud_ml_sdk._types.operation:110] Operation fvtcuk5pj2erssq21335 have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 05:12:24 yandex_cloud_ml_sdk._types.operation:110] Operation fvtcuk5pj2erssq21335 have non-terminal status IN_PROGRESS, sleep for 0.500000s
[D 2025-04-02 05:12:25 yandex_cloud_ml_sdk._types.operation:1

Для стейка я могу порекомендовать несколько отличных вариантов вин:

1. **Для нежного мраморного стейка (Филе-миньон)** подойдут легкие и элегантные красные вина из винограда Пино Нуар, Нерелло Маскалезе, а также выдержанное и шелковистое Мерло. Хороши будут "округлые", выдержанные варианты из сортов Неббиоло (например, Барбареско), Темпранильо (из Рибейра дель Дуэро) и Санджовезе (например, Кьянти Ризерва).

2. **Для жирноватого мраморного стейка (Рибай и подобные)**:
   - При прожарке Rare — выдержанные и "благородные" вина из Темпранильо (например, из Рибейра дель Дуэро или любые от Ризервы и выше), Санджовезе (Кьянти Ризерва, Брунелло), "супертосканские" вина, Бордо Правого берега, а также шелковистые аргентинские Мальбеки.
   - При прожарке Medium или WellDone — сухие и полусухие из винограда Сира (Шираз), Каберне Совиньон, "тельный" Мальбек, Примитиво, Зинфандель, Альянико (выдержанное и слегка "округлившееся"), выдержанный "ронский" ассамбляж Гренаш+Сира+Мурведр, вина Приората от 6-8 лет выдержки и выше.

Если вас заинтересовал какой-либо из этих вариантов, я могу помочь найти его в нашем прайс-листе и даже добавить в корзину. Какой вариант вам нравится больше?

## Делаем винного ассистента в телеграме

Для демонстрации возможностей бота мы реализуем такого ассистента в виде телеграм-бота. Конечно, для реализации полноценного телеграм-бота необходимо использовать виртуальную машину и режим webhooks, но в нашем случае мы ограничимся режимом поллинга, и запустим бота прямо в Datasphere.

> Прежде, чем запускать код ниже, необходимо создать чат-бота, пообщавшись с [@botfather](http://t.me/botfather), и разместить его секрет в виде секрета в Datasphere.

Для начала установим необходимую библиотеку:

In [103]:
%pip install --quiet telebot

I0000 00:00:1743514967.984864    3491 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [121]:
import telebot

telegram_token = os.environ["tg_token"]

bot = telebot.TeleBot(telegram_token)

threads = {}


def get_thread(chat_id):
    if chat_id in threads.keys():
        return threads[chat_id]
    t = create_thread()
    print(f"New thread {t.id=} created")
    threads[chat_id] = t
    return t


# Обработчик команды /start
@bot.message_handler(commands=["start"])
def start(message):
    t = get_thread(message.chat.id)
    print(f"Starting on thread {t.id=}, msg={message.text}")
    ans = wine_agent(message.text, thread=t)
    bot.send_message(message.chat.id, ans)


# Обработчик для всех входящих сообщений
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    t = get_thread(message.chat.id)
    print(f"Answering on thread {t.id=}, msg={message.text}")
    answer = wine_agent(message.text, thread=t)
    bot.send_message(message.chat.id, answer)


# Запуск бота
print("Бот готов к работе")
bot.polling(none_stop=True)

Бот готов к работе
New thread t.id='fvtalu3ssh0vc0k7qq4q' created
Starting on thread t.id='fvtalu3ssh0vc0k7qq4q', msg=/start
Answering on thread t.id='fvtalu3ssh0vc0k7qq4q', msg=Какое самое дешевое вино?
Вызываем функцию SearchWinePriceList, args=name=None country=None acidity=None color=None sort_order='cheapest' what_to_return='price'
Answering on thread t.id='fvtalu3ssh0vc0k7qq4q', msg=Положи вино адгум в корзину
Вызываем функцию AddToCart, args=wine_name='ВИНО АДАГУМ КБСВ КР СХ' count='1'
Answering on thread t.id='fvtalu3ssh0vc0k7qq4q', msg=Что в корзине?
Вызываем функцию ShowCart, args=
Answering on thread t.id='fvtalu3ssh0vc0k7qq4q', msg=Вызови оператора
Вызываем функцию Handover, args=reason='Пользователь хочет вызвать оператора'


## Удаляем лишнее

В заключение удалим созданные ресурсы:

In [79]:
wine_agent.done(delete_assistant=True)
index.delete()
for f in df['Uploaded']:
    f.delete()
for f in uploaded_foodwine:
    f.delete()

Если вы хотите на всякий случай удалить другие оставшиеся ресурсы, то можете использовать код ниже - он удалит все переписки, файлы, индексы и ассистенты. 

**ВНИМАНИЕ**: Не выполняйте этот код, если у вас есть другие проекты с ассистентами в облаке!

In [80]:
for thread in sdk.threads.list():
    print(f" + deleting thread id={thread.id}")
    try:
        thread.delete()
    except:
        print(" ! Error")
for assistant in sdk.assistants.list():
    print(f" + deleting assistant id={assistant.id}")
    assistant.delete()

 + deleting thread id=fvthlaradjqr37src6o8
 + deleting thread id=fvt22afaeuhoefth43ij
 + deleting thread id=fvttkc8l9q0inhabdjvl
 + deleting thread id=fvtmi1sdkg9bv34fatdf
 ! Error
 + deleting thread id=fvtgchjftun6cthrfcs3
 ! Error
 + deleting thread id=fvt0fk0h42mf50ofin0b
 ! Error
 + deleting thread id=fvt2a02a0cleq9gk3j75
 ! Error
 + deleting thread id=fvt4iugnrk28pvtlk76t
 ! Error
 + deleting thread id=fvtiuhempeslu1i7qtnu
 ! Error
 + deleting thread id=fvtmubvkqhtfrqcsjduo
 ! Error
 + deleting thread id=fvtjfjds1aoo6f0r4ptj
 ! Error
 + deleting thread id=fvtaljptt0kjjuggnm93
 ! Error
 + deleting thread id=fvtsp56titc7hdtrkv54
 ! Error
 + deleting thread id=fvtm57amjtsq8gr12f6t
 ! Error
 + deleting thread id=fvtghqr56rcu9d80qs3k
 ! Error
 + deleting thread id=fvtk8gq8q6vrf22mkdto
 ! Error
 + deleting thread id=fvth7obnf78rmksv2ptg
 ! Error
 + deleting thread id=fvt59jcebc5san8tv4hl
 ! Error
 + deleting thread id=fvtrnoed344uq8ssahae
 ! Error
 + deleting thread id=fvtaurmqfovnlaifq

In [81]:
from tqdm.auto import tqdm

for index in sdk.search_indexes.list():
    print(f" + deleting index id={index.id}")
    index.delete()
print(' + Deleting files')
for file in tqdm(sdk.files.list()):
    #print(f" + deleting file id={file.id}")
    file.delete()

 + deleting index id=fvtlk0391ajbl4i88t4g
 + deleting index id=fvtbu1qgiqf3pmgq15hr
 + Deleting files


304it [00:17, 17.02it/s]
